In [29]:
from itertools import chain
import os
from pathlib import Path
import re
import json
from collections import OrderedDict
from copy import deepcopy

import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import pydot
import networkx
from IPython.display import SVG, display, HTML
import IPython

pd.set_option('display.max_colwidth', 500)

In [33]:
COLOR_SCHEME = dict(
    background='#242424',
    node='#343434',
    node_fontcolor='#B6ECF3',
    node_value='#343434',
    node_value_fontcolor='#DE958E',
    edge_color='#B6ECF3',
    edge_value_color='#DE958E',
    node_library_font='#DE958E',
    node_subpackage_font='#A0D17B',
    node_module_font='#B6ECF3',
    edge_library='#DE958E',
    edge_subpackage='#A0D17B',
    edge_module='#B6ECF3',
)

In [34]:
def is_iterable(item):
    for type_ in [list, tuple, set, dict, OrderedDict]:
        if isinstance(item, type_):
            return True
    return False

def is_dictlike(item):
    for type_ in [dict, OrderedDict]:
        if isinstance(item, type_):
            return True
    return False

def is_listlike(item):
    for type_ in [list, tuple, set]:
        if isinstance(item, type_):
            return True
    return False

def flatten(item, separator='/', embed_types=True):
    output = {}
    def recurse(item, cursor):
        if is_listlike(item):
            if embed_types:
                name = item.__class__.__name__
                item = [(f'<{name}_{i}>', val) for i, val in enumerate(item)]
                item = dict(item)
            else:
                item = dict(enumerate(item))
        if is_dictlike(item):
            for key, val in item.items():
                new_key = f'{cursor}{separator}{str(key)}'
                if is_iterable(val) and len(val) > 0:
                    recurse(val, new_key)
                else:
                    output[new_key] = val

    recurse(item, '')
    return output

def nest(dict_, separator='/'):
    output = {}
    for keys, val in dict_.items():
        split_keys = list(filter(
            lambda x: x != '', keys.split(separator)
        ))
        cursor = output
        last = split_keys.pop()
        for key in split_keys:
            if key not in cursor:
                cursor[key] = {}
            cursor = cursor[key]
        cursor[last] = val
    return output

def unembed(item):
    lut = {'list': list, 'tuple': tuple, 'set': set}
    embed_re = re.compile('^<([a-z]+)_(\d+)>$')
    
    output = {}
    if is_dictlike(item) and item != {}:
        keys = list(item.keys())

        match = embed_re.match(keys[0])
        if match:
            indices = [embed_re.match(key).group(2) for key in keys]
            indices = map(int, indices)

            output = []
            for i, key in sorted(zip(indices, keys)):
                next_item = item[key]
                if is_dictlike(next_item):
                    next_item = unembed(next_item)
                output.append(next_item)
            
            output = lut[match.group(1)](output)
            return output
        else:
            for key, val in item.items():
                output[key] = unembed(val)
    else:
        output = item
    return output

def edit(dict_, regex, key_func=None, val_func=None):
    if key_func is None:
        key_func = lambda k: k
    if val_func is None:
        val_func = lambda v: v
    
    output = {}
    for key, val in dict_.items():
        if re.search(regex, key):
            output[key_func(key)] = val_func(val)
        else:
            output[key] = val
    return output

def is_number(item):
    if isinstance(item, str):
        if re.search('\d:', item):
            return True
    if isinstance(item, int):
        return True
    if isinstance(item, float):
        return True
    return False

def unit_string_to_float(string):
    lut = dict(
        millimeter=1 / 1000000,
        microliter=1 / 100
    )
    num, unit = string.split(':')
    output = float(num) * lut[unit]
    return output

def list_all_files(directory):
    output = []
    for root, dirs, files in os.walk(directory):
        for file_ in files:
            fullpath = Path(root, file_)
            output.append(fullpath)
    return output

def get_parents(item, separator='.'):
    items = item.split(separator)
    output = []
    for i in range(len(items) - 1):
        output.append(separator.join(items[:i+1]))
    return output

def drop_duplicates(items):
    temp = set()
    output = []
    for item in items:
        if item not in temp:
            output.append(item)
            temp.add(item)
    return output

def get_imports(fullpath):
    with open(fullpath) as f:
        data = f.readlines()
    data = map(lambda x: x.strip('\n'), data)
    data = filter(lambda x: re.search('^import|^from', x), data)
    data = map(lambda x: re.sub('from (.*?) .*', '\\1', x), data)
    data = map(lambda x: re.sub(' as .*', '', x), data)
    data = map(lambda x: re.sub(' *#.*', '', x), data)
    data = map(lambda x: re.sub('import ', '', x), data)
    data = filter(lambda x: not is_builtin(x), data)
    return list(data)

def is_builtin(item):
    builtins = [
        'copy',
        'datetime',
        'enum',
        'functools',
        'inspect',
        'itertools',
        'json',
        'logging',
        'math',
        'os',
        'pathlib',
        're',
        'uuid'
    ]
    suffix = '(\..*)?'
    builtins_re = f'{suffix}|'.join(builtins) + suffix
    if re.search(builtins_re, item):
        return True
    return False

def dot_to_html(dot, layout='dot'):
    layouts = ['dot', 'twopi', 'circo', 'neato', 'fdp', 'sfdp']
    if layout not in layouts:
        msg = f'Invalid layout value. {layout} not in {layouts}.'
        raise ValueError(msg)

    svg = dot.create_svg(prog=layout)
    html = f'<img width="100%" src="data:image/svg+xml;{svg}" >'
    return HTML(html)

def write_dot_graph(self, graph, fullpath, layout='dot', orthogonal_edges=False, color_scheme=COLOR_SCHEME):
    if isinstance(fullpath, Path):
        fullpath = Path(fullpath).absolute().as_posix()

    _, ext = os.path.splitext(fullpath)
    if re.search('\.svg$', ext, re.I):
        graph.write_svg(fullpath)
    elif re.search('\.dot$', ext, re.I):
        graph.write_dot(fullpath)
    elif re.search('\.png$', ext, re.I):
        graph.write_png(fullpath)
    else:
        msg = f'Invalid extension found: {ext}. '
        msg += 'Valid extensions include: svg, dot, png.'
        raise ValueError(msg)

    return self

In [38]:
class BlobETL():
    def __init__(self, item, separator='/'):
        self._data = flatten(item, separator=separator, embed_types=True)
        self._separator = separator
    
    def to_dict(self):
        return unembed(nest(self._data))
    
    def to_flat_dict(self):
        return self._data
    
    def filter(self, predicate, by='key'):
        data = {}
        if by not in ['key', 'value', 'key+value']:
            msg = f'Invalid by argument: {by}. Needs to be one of: '
            msg += 'key, value, key+value.'
            raise ValueError(msg)

        for key, val in self._data.items():
            item = None
            if by == 'key':
                item = key
            elif by == 'value':
                item = val
            else:
                item = [key, val]
                
            if predicate(item):
                data[key] = val

        self._data = data
        return self
        
    def delete(self, predicate, by='key'):
        data = deepcopy(self._data)
        if by not in ['key', 'value', 'key+value']:
            msg = f'Invalid by argument: {by}. Needs to be one of: '
            msg += 'key, value, key+value.'
            raise ValueError(msg)

        for key, val in self._data.items():
            item = None
            if by == 'key':
                item = key
            elif by == 'value':
                item = val
            else:
                item = [key, val]
                
            if predicate(item):
                del data[key]

        self._data = data
        return self
    
    def set(
        self,
        predicate=None,
        key_setter=None,
        value_setter=None,
        by='key'
    ):
        if by not in ['key', 'value', 'key+value']:
            msg = f'Invalid by argument: {by}. Needs to be one of: '
            msg += 'key, value, key+value.'
            raise ValueError(msg)
            
        if predicate is None:
            predicate = lambda x: x
        
        if key_setter is None:
            key_setter = lambda x: x
        
        if value_setter is None:
            value_setter = lambda x: x
        
        data = deepcopy(self._data)
        for key, val in self._data.items():
            item = None
            if by == 'key':
                item = key
            elif by == 'value':
                item = val
            else:
                item = [key, val]
                
            if predicate(item):
                k = key_setter(key)
                v = value_setter(val)
                del data[key]
                data[k] = v

        self._data = data
        return self
    
    def update(self, dict_):
        data = flatten(dict_, separator=separator, embed_types=True)
        self._data.update(data)
        return self

    def to_networkx_graph(self):
        graph = networkx.DiGraph()
        graph.add_node('root')
        embed_re = re.compile('<[a-z]+_(\d+)>')

        def recurse(item, parent):
            for key, val in item.items():
                k = f'{parent}{self._separator}{key}'
                short_name = embed_re.sub('\\1', key)
                graph.add_node(k, short_name=short_name, node_type='key')
                graph.add_edge(parent, k)

                if isinstance(val, dict):
                    recurse(val, k)
                else:
                    graph.nodes[k]['value'] = [val]
                    name = f'"{str(val)}"'
                    v = f'"{k}{self._separator}{str(val)}"'
                    graph.add_node(v, short_name=name, node_type='value')
                    graph.add_edge(k, v)

        recurse(nest(self._data), 'root')
        graph.remove_node('root')
        return graph

    def to_dot_graph(self, orthogonal_edges=False, color_scheme=COLOR_SCHEME):
        graph = self.to_networkx_graph()
        dot = networkx.drawing.nx_pydot.to_pydot(graph)

        dot.set_bgcolor(color_scheme['background'])
        if orthogonal_edges:
            dot.set_splines('ortho')

        for node in dot.get_nodes():
            node.set_shape('rect')
            node.set_style('filled')
            node.set_color(color_scheme['node'])
            node.set_fillcolor(color_scheme['node'])
            node.set_fontcolor(color_scheme['node_fontcolor'])
            node.set_fontname('Courier')

            attrs = node.get_attributes()
            if 'short_name' in attrs:
                node.set_label(attrs['short_name'])
            if 'node_type' in attrs and attrs['node_type'] == 'value':
                node.set_color(color_scheme['node_value'])
                node.set_fillcolor(color_scheme['node_value'])
                node.set_fontcolor(color_scheme['node_value_fontcolor'])

        for edge in dot.get_edges():
            edge.set_color(color_scheme['edge_color'])

            node = dot.get_node(edge.get_destination())[0]
            attrs = node.get_attributes()
            if 'node_type' in attrs and attrs['node_type'] == 'value':
                edge.set_color(color_scheme['edge_value_color'])

        return dot
        
    def to_html(self, layout='dot', orthogonal_edges=False, color_scheme=COLOR_SCHEME):
        dot = self.to_dot_graph(
            orthogonal_edges=orthogonal_edges,
            color_scheme=color_scheme,
        )
        return dot_to_html(dot, layout=layout)
    
    def write(
        self,
        fullpath,
        layout='dot',
        orthogonal_edges=False,
        color_scheme=COLOR_SCHEME
    ):
        write_dot_graph(
            fullpath,
            layout=layout,
            orthogonal_edges=orthogonal_edges,
            color_scheme=color_scheme,
        )
        return self

In [39]:
class DependencyETL():
    def __init__(
        self,
        source,
        include_regex='.*\.py$',
        exclude_regex='_test\.py$',
    ):
        self._source = source
        self._data = DependencyETL._get_data(source, include_regex, exclude_regex)
        
    @staticmethod
    def _get_data(source, include_regex, exclude_regex):
        files = list_all_files(source)
        if include_regex != '':
            if not include_regex.endswith('\.py$'):
                msg = 'include_regex does not end in "\.py$".'
                raise ValueError(msg)

            files = filter(lambda x: re.search(include_regex, x.absolute().as_posix()), files)
        if exclude_regex != '':
            files = filter(lambda x: not re.search(exclude_regex, x.absolute().as_posix()), files)
        files = list(files)

        # buid DataFrame of nodes and imported dependencies
        data = DataFrame()
        data['fullpath'] = files
        data['node_name'] = data.fullpath\
            .apply(lambda x: x.absolute().as_posix())\
            .apply(lambda x: re.sub(source, '', x))\
            .apply(lambda x: re.sub('\.py$', '', x))\
            .apply(lambda x: re.sub('^/', '', x))\
            .apply(lambda x: re.sub('/', '.', x))

        data['subpackages'] = data.node_name.apply(get_parents).apply(drop_duplicates)
        data.subpackages = data.subpackages.apply(lambda x: list(filter(lambda y: y != '', x)))

        data['dependencies'] = data.fullpath.apply(get_imports).apply(drop_duplicates)
        data.dependencies += data.node_name.apply(lambda x: ['.'.join(x.split('.')[:-1])])
        data.dependencies = data.dependencies.apply(lambda x: list(filter(lambda y: y != '', x)))

        data['node_type'] = 'module'

        # add subpackages as nodes
        pkgs = set(chain(*data.subpackages.tolist()))
        pkgs = pkgs.difference(data.node_name.tolist())
        pkgs = sorted(list(pkgs))
        pkgs = Series(pkgs)\
            .apply(
                lambda x: dict(
                    node_name=x,
                    node_type='subpackage',
                    dependencies=get_parents(x),
                    subpackages=get_parents(x),
                )
            ).tolist()
        pkgs = DataFrame(pkgs)
        data = data.append(pkgs, ignore_index=True, sort=True)

        # add library dependencies as nodes
        libs = set(chain(*data.dependencies.tolist()))
        libs = libs.difference(data.node_name.tolist())
        libs = sorted(list(libs))
        libs = Series(libs)\
            .apply(
                lambda x: dict(
                    node_name=x,
                    node_type='library',
                    dependencies=[],
                    subpackages=[],
                )
            ).tolist()
        libs = DataFrame(libs)
        data = data.append(libs, ignore_index=True, sort=True)

        data.drop_duplicates('node_name', inplace=True)
        data.reset_index(drop=True, inplace=True)

        # define node coordinates
        data['x'] = 0
        data['y'] = 0
        data = DependencyETL._calculate_coordinates(data)
        data = DependencyETL._anneal_coordinates(data)
        data = DependencyETL._center_x_coordinates(data)

        cols = [
            'node_name',
            'node_type',
            'x',
            'y',
            'dependencies',
            'subpackages',
            'fullpath',
        ]
        data = data[cols]
        return data
        
    @staticmethod
    def _calculate_coordinates(data, epochs=10):
        for item in ['module', 'subpackage', 'library']:
            mask = data.node_type == item
            n = data[mask].shape[0]

            index = data[mask].index
            data.loc[index, 'x'] = list(range(n))

            if item != 'library':
                data.loc[index, 'y'] = data.loc[index, 'node_name']\
                    .apply(lambda x: len(x.split('.')))

        max_ = data[data.node_type == 'subpackage'].y.max()
        index = data[data.node_type == 'module'].index
        data.loc[index, 'y'] += max_
        data.loc[index, 'y'] += data.loc[index, 'subpackages'].apply(len)

        # reverse y
        max_ = data.y.max()
        data.y = -1 * data.y + max_

        return data
    
    @staticmethod
    def _anneal_coordinates(data, epochs=10):
        for epoch in range(epochs):
            graph = DependencyETL._to_networkx_graph(data)
            if epoch % 2 == 0:
                graph = graph.reverse()

            for name in graph.nodes:
                tree = networkx.bfs_tree(graph, name)
                mu_x = np.mean([graph.nodes[n]['x'] for n in tree])
                graph.nodes[name]['x'] = mu_x

            for node in graph.nodes:
                mask = data[data.node_name == node].index
                data.loc[mask, 'x'] = graph.nodes[node]['x']

            # rectify node x coordinates
            data.sort_values('x', inplace=True)
            for y in data.y.unique():
                mask = data[data.y == y].index
                values = data.loc[mask, 'x'].tolist()
                values = list(range(len(values)))
                data.loc[mask, 'x'] = values

        return data
    
    @staticmethod
    def _center_x_coordinates(data):
        max_ = data.x.max()
        for y in data.y.unique():
            mask = data[data.y == y].index
            l_max = data.loc[mask, 'x'].max()
            delta = max_ - l_max
            data.loc[mask, 'x'] += (delta / 2)
        return data

    @staticmethod
    def _to_networkx_graph(data):
        graph = networkx.DiGraph()
        data.apply(
            lambda x: graph.add_node(
                x.node_name,
                **{k: getattr(x, k) for k in x.index}
            ),
            axis=1
        )

        data.apply(lambda x: [graph.add_edge(p, x.node_name) for p in x.dependencies], axis=1)
        return graph
    
    def to_networkx_graph(self):
        return DependencyETL._to_networkx_graph(self._data)
    
    def to_dot_graph(
        self,
        orthogonal_edges=False,
        color_scheme=COLOR_SCHEME,
    ):
        graph = self.to_networkx_graph()
        dot = networkx.drawing.nx_pydot.to_pydot(graph)
        if orthogonal_edges:
            dot.set_splines('ortho')
        dot.set_bgcolor(color_scheme['background'])

        for node in dot.get_nodes():
            gnode = re.sub('"', '', node.get_name())
            gnode = graph.nodes[gnode]
            if gnode == {}:
                continue

            node.set_pos(f"{gnode['x']},{gnode['y']}!")

            node.set_shape('rect')
            node.set_style('filled')
            node.set_color(color_scheme['node'])
            node.set_fillcolor(color_scheme['node'])
            if gnode['node_type'] == 'library':
                node.set_fontcolor(color_scheme['node_library_font'])
            elif gnode['node_type'] == 'subpackage':
                node.set_fontcolor(color_scheme['node_subpackage_font'])
            else:
                node.set_fontcolor(color_scheme['node_module_font'])
            node.set_fontname('Courier')

        for edge in dot.get_edges():
            gnode = dot.get_node(edge.get_source())
            gnode = gnode[0].get_name()
            gnode = re.sub('"', '', gnode)
            gnode = graph.nodes[gnode]
            if gnode == {}:
                continue

            if gnode['node_type'] == 'library':
                edge.set_color(color_scheme['edge_library'])
            elif gnode['node_type'] == 'subpackage':
                edge.set_color(color_scheme['edge_subpackage'])
            else:
                edge.set_color(color_scheme['edge_module'])

        return dot
    
    def to_dataframe(self):
        return self._data
    
    def to_html(self, layout='dot', orthogonal_edges=False, color_scheme=COLOR_SCHEME):
        dot = self.to_dot_graph(
            orthogonal_edges=orthogonal_edges,
            color_scheme=color_scheme,
        )
        return dot_to_html(dot, layout=layout)
    
    def write(
        self,
        fullpath,
        layout='dot',
        orthogonal_edges=False,
        color_scheme=COLOR_SCHEME
    ):
        write_dot_graph(
            fullpath,
            layout=layout,
            orthogonal_edges=orthogonal_edges,
            color_scheme=color_scheme,
        )
        return self

In [40]:
source = '/root/rolling-pin/python'
d = DependencyETL(source)
d.to_html()

In [45]:
x = '{"shape": {"rows": 1, "columns": 1, "format": "SBS96"}, "op": "liquid_handle", "locations": [{"transports": [{"mode_params": {"tip_position": {"position_z": {"reference": "well_top", "offset": "1:millimeter"}}}}, {"volume": "-5:microliter", "mode_params": {"liquid_class": "air", "tip_position": {"position_z": {"reference": "preceding_position"}}}}, {"mode_params": {"tip_position": {"position_z": {"reference": "well_top"}}}}, {"mode_params": {"tip_position": {"position_z": {"detection": {"method": "capacitance"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"volume": "-5:microliter", "pump_override_volume": "-5:microliter", "mode_params": {"tip_position": {"position_z": {"detection": {"method": "tracked"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"volume": "-3.21:microliter", "mode_params": {"tip_position": {"position_z": {"detection": {"method": "tracked"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"volume": "5:microliter", "pump_override_volume": "5:microliter", "mode_params": {"tip_position": {"position_z": {"detection": {"method": "tracked"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"mode_params": {"tip_position": {"position_z": {"reference": "well_top"}}}}, {"volume": "-2:microliter", "mode_params": {"liquid_class": "air", "tip_position": {"position_z": {"reference": "preceding_position"}}}}], "location": "Inducer-Solvent Source/0"}, {"transports": [{"mode_params": {"tip_position": {"position_z": {"reference": "well_top"}}}}, {"volume": "2:microliter", "mode_params": {"liquid_class": "air", "tip_position": {"position_z": {"reference": "preceding_position"}}}}, {"mode_params": {"tip_position": {"position_z": {"reference": "well_top"}}}}, {"mode_params": {"tip_position": {"position_z": {"detection": {"method": "capacitance"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"volume": "3.21:microliter", "mode_params": {"tip_position": {"position_z": {"detection": {"method": "tracked"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"mode_params": {"tip_position": {"position_z": {"reference": "well_top"}}}}, {"mode_params": {"tip_position": {"position_z": {"detection": {"method": "capacitance"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"volume": "-3.21:microliter", "mode_params": {"tip_position": {"position_z": {"detection": {"method": "tracked"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"volume": "3.21:microliter", "mode_params": {"tip_position": {"position_z": {"detection": {"method": "tracked"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"volume": "-3.21:microliter", "mode_params": {"tip_position": {"position_z": {"detection": {"method": "tracked"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"volume": "3.21:microliter", "mode_params": {"tip_position": {"position_z": {"detection": {"method": "tracked"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"volume": "-3.21:microliter", "mode_params": {"tip_position": {"position_z": {"detection": {"method": "tracked"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"volume": "3.21:microliter", "mode_params": {"tip_position": {"position_z": {"detection": {"method": "tracked"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"volume": "-3.21:microliter", "mode_params": {"tip_position": {"position_z": {"detection": {"method": "tracked"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"volume": "3.21:microliter", "mode_params": {"tip_position": {"position_z": {"detection": {"method": "tracked"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"volume": "-3.21:microliter", "mode_params": {"tip_position": {"position_z": {"detection": {"method": "tracked"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"volume": "3.21:microliter", "mode_params": {"tip_position": {"position_z": {"detection": {"method": "tracked"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"volume": "-3.21:microliter", "mode_params": {"tip_position": {"position_z": {"detection": {"method": "tracked"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"volume": "3.21:microliter", "mode_params": {"tip_position": {"position_z": {"detection": {"method": "tracked"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"volume": "-3.21:microliter", "mode_params": {"tip_position": {"position_z": {"detection": {"method": "tracked"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"volume": "3.21:microliter", "mode_params": {"tip_position": {"position_z": {"detection": {"method": "tracked"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"volume": "-3.21:microliter", "mode_params": {"tip_position": {"position_z": {"detection": {"method": "tracked"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"volume": "3.21:microliter", "mode_params": {"tip_position": {"position_z": {"detection": {"method": "tracked"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"volume": "-3.21:microliter", "mode_params": {"tip_position": {"position_z": {"detection": {"method": "tracked"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"volume": "3.21:microliter", "mode_params": {"tip_position": {"position_z": {"detection": {"method": "tracked"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"volume": "-3.21:microliter", "mode_params": {"tip_position": {"position_z": {"detection": {"method": "tracked"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"volume": "3.21:microliter", "mode_params": {"tip_position": {"position_z": {"detection": {"method": "tracked"}, "reference": "liquid_surface", "offset": "-1:millimeter"}}}}, {"mode_params": {"tip_position": {"position_z": {"reference": "well_top"}}}}, {"volume": "5:microliter", "mode_params": {"liquid_class": "air", "tip_position": {"position_z": {"reference": "preceding_position"}}}}], "location": "Inducer Reservoir/2"}], "mode": "air_displacement"}'
x = json.loads(x)

leafs = ['liquid_class', 'reference', 'volume', 'method', 'offset']
for leaf in leafs:
    b = BlobETL(x)
    b = b.filter(lambda x: re.search('0>/transports', x))
    b = b.filter(lambda x: re.search(leaf, x))
    html = b.to_html(layout='dot')
    IPython.display.display_html(html)

\n<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"\n "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">\n<!-- Generated by graphviz version 2.40.1 (20161225.0304)\n -->\n<!-- Title: %3 Pages: 1 -->\n \n \n %3 \n \n<!-- root/locations -->\n \n root/locations \n \n locations \n \n<!-- root/locations/<list_0> -->\n \n root/locations/<list_0> \n \n 0 \n \n<!-- root/locations->root/locations/<list_0> -->\n \n root/locations->root/locations/<list_0> \n \n \n \n<!-- root/locations/<list_0>/transports -->\n \n root/locations/<list_0>/transports \n \n transports \n \n<!-- root/locations/<list_0>->root/locations/<list_0>/transports -->\n \n root/locations/<list_0>->root/locations/<list_0>/transports \n \n \n \n<!-- root/locations/<list_0>/transports/<list_1> -->\n \n root/locations/<list_0>/transports/<list_1> \n \n 1 \n \n<!-- root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_1> -->\n \n root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_1> \n \n \n \n<!-- root/locations/<list_0>/transports/<list_8> -->\n \n root/locations/<list_0>/transports/<list_8> \n \n 8 \n \n<!-- root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_8> -->\n \n root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_8> \n \n \n \n<!-- root/locations/<list_0>/transports/<list_1>/mode_params -->\n \n root/locations/<list_0>/transports/<list_1>/mode_params \n \n mode_params \n \n<!-- root/locations/<list_0>/transports/<list_1>->root/locations/<list_0>/transports/<list_1>/mode_params -->\n \n root/locations/<list_0>/transports/<list_1>->root/locations/<list_0>/transports/<list_1>/mode_params \n \n \n \n<!-- root/locations/<list_0>/transports/<list_1>/mode_params/liquid_class -->\n \n root/locations/<list_0>/transports/<list_1>/mode_params/liquid_class \n \n liquid_class \n \n<!-- root/locations/<list_0>/transports/<list_1>/mode_params->root/locations/<list_0>/transports/<list_1>/mode_params/liquid_class -->\n \n root/locations/<list_0>/transports/<list_1>/mode_params->root/locations/<list_0>/transports/<list_1>/mode_params/liquid_class \n \n \n \n<!-- root/locations/<list_0>/transports/<list_1>/mode_params/liquid_class/air -->\n \n root/locations/<list_0>/transports/<list_1>/mode_params/liquid_class/air \n \n air \n \n<!-- root/locations/<list_0>/transports/<list_1>/mode_params/liquid_class->root/locations/<list_0>/transports/<list_1>/mode_params/liquid_class/air -->\n \n root/locations/<list_0>/transports/<list_1>/mode_params/liquid_class->root/locations/<list_0>/transports/<list_1>/mode_params/liquid_class/air \n \n \n \n<!-- root/locations/<list_0>/transports/<list_8>/mode_params -->\n \n root/locations/<list_0>/transports/<list_8>/mode_params \n \n mode_params \n \n<!-- root/locations/<list_0>/transports/<list_8>->root/locations/<list_0>/transports/<list_8>/mode_params -->\n \n root/locations/<list_0>/transports/<list_8>->root/locations/<list_0>/transports/<list_8>/mode_params \n \n \n \n<!-- root/locations/<list_0>/transports/<list_8>/mode_params/liquid_class -->\n \n root/locations/<list_0>/transports/<list_8>/mode_params/liquid_class \n \n liquid_class \n \n<!-- root/locations/<list_0>/transports/<list_8>/mode_params->root/locations/<list_0>/transports/<list_8>/mode_params/liquid_class -->\n \n root/locations/<list_0>/transports/<list_8>/mode_params->root/locations/<list_0>/transports/<list_8>/mode_params/liquid_class \n \n \n \n<!-- root/locations/<list_0>/transports/<list_8>/mode_params/liquid_class/air -->\n \n root/locations/<list_0>/transports/<list_8>/mode_params/liquid_class/air \n \n air \n \n<!-- root/locations/<list_0>/transports/<list_8>/mode_params/liquid_class->root/locations/<list_0>/transports/<list_8>/mode_params/liquid_class/air -->\n \n root/locations/<list_0>/transports/<list_8>/mode_params/liquid_class->root/locations/<list_0>/transports/<list_8>/mode_params/liquid_class/air \n \n \n \n \n \n'" >

\n<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"\n "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">\n<!-- Generated by graphviz version 2.40.1 (20161225.0304)\n -->\n<!-- Title: %3 Pages: 1 -->\n \n \n %3 \n \n<!-- root/locations -->\n \n root/locations \n \n locations \n \n<!-- root/locations/<list_0> -->\n \n root/locations/<list_0> \n \n 0 \n \n<!-- root/locations->root/locations/<list_0> -->\n \n root/locations->root/locations/<list_0> \n \n \n \n<!-- root/locations/<list_0>/transports -->\n \n root/locations/<list_0>/transports \n \n transports \n \n<!-- root/locations/<list_0>->root/locations/<list_0>/transports -->\n \n root/locations/<list_0>->root/locations/<list_0>/transports \n \n \n \n<!-- root/locations/<list_0>/transports/<list_0> -->\n \n root/locations/<list_0>/transports/<list_0> \n \n 0 \n \n<!-- root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_0> -->\n \n root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_0> \n \n \n \n<!-- root/locations/<list_0>/transports/<list_1> -->\n \n root/locations/<list_0>/transports/<list_1> \n \n 1 \n \n<!-- root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_1> -->\n \n root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_1> \n \n \n \n<!-- root/locations/<list_0>/transports/<list_2> -->\n \n root/locations/<list_0>/transports/<list_2> \n \n 2 \n \n<!-- root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_2> -->\n \n root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_2> \n \n \n \n<!-- root/locations/<list_0>/transports/<list_3> -->\n \n root/locations/<list_0>/transports/<list_3> \n \n 3 \n \n<!-- root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_3> -->\n \n root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_3> \n \n \n \n<!-- root/locations/<list_0>/transports/<list_4> -->\n \n root/locations/<list_0>/transports/<list_4> \n \n 4 \n \n<!-- root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_4> -->\n \n root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_4> \n \n \n \n<!-- root/locations/<list_0>/transports/<list_5> -->\n \n root/locations/<list_0>/transports/<list_5> \n \n 5 \n \n<!-- root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_5> -->\n \n root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_5> \n \n \n \n<!-- root/locations/<list_0>/transports/<list_6> -->\n \n root/locations/<list_0>/transports/<list_6> \n \n 6 \n \n<!-- root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_6> -->\n \n root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_6> \n \n \n \n<!-- root/locations/<list_0>/transports/<list_7> -->\n \n root/locations/<list_0>/transports/<list_7> \n \n 7 \n \n<!-- root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_7> -->\n \n root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_7> \n \n \n \n<!-- root/locations/<list_0>/transports/<list_8> -->\n \n root/locations/<list_0>/transports/<list_8> \n \n 8 \n \n<!-- root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_8> -->\n \n root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_8> \n \n \n \n<!-- root/locations/<list_0>/transports/<list_0>/mode_params -->\n \n root/locations/<list_0>/transports/<list_0>/mode_params \n \n mode_params \n \n<!-- root/locations/<list_0>/transports/<list_0>->root/locations/<list_0>/transports/<list_0>/mode_params -->\n \n root/locations/<list_0>/transports/<list_0>->root/locations/<list_0>/transports/<list_0>/mode_params \n \n \n \n<!-- root/locations/<list_0>/transports/<list_0>/mode_params/tip_position -->\n \n root/locations/<list_0>/transports/<list_0>/mode_params/tip_position \n \n tip_position \n \n<!-- root/locations/<list_0>/transports

\n<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"\n "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">\n<!-- Generated by graphviz version 2.40.1 (20161225.0304)\n -->\n<!-- Title: %3 Pages: 1 -->\n \n \n %3 \n \n<!-- root/locations -->\n \n root/locations \n \n locations \n \n<!-- root/locations/<list_0> -->\n \n root/locations/<list_0> \n \n 0 \n \n<!-- root/locations->root/locations/<list_0> -->\n \n root/locations->root/locations/<list_0> \n \n \n \n<!-- root/locations/<list_0>/transports -->\n \n root/locations/<list_0>/transports \n \n transports \n \n<!-- root/locations/<list_0>->root/locations/<list_0>/transports -->\n \n root/locations/<list_0>->root/locations/<list_0>/transports \n \n \n \n<!-- root/locations/<list_0>/transports/<list_1> -->\n \n root/locations/<list_0>/transports/<list_1> \n \n 1 \n \n<!-- root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_1> -->\n \n root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_1> \n \n \n \n<!-- root/locations/<list_0>/transports/<list_4> -->\n \n root/locations/<list_0>/transports/<list_4> \n \n 4 \n \n<!-- root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_4> -->\n \n root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_4> \n \n \n \n<!-- root/locations/<list_0>/transports/<list_5> -->\n \n root/locations/<list_0>/transports/<list_5> \n \n 5 \n \n<!-- root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_5> -->\n \n root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_5> \n \n \n \n<!-- root/locations/<list_0>/transports/<list_6> -->\n \n root/locations/<list_0>/transports/<list_6> \n \n 6 \n \n<!-- root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_6> -->\n \n root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_6> \n \n \n \n<!-- root/locations/<list_0>/transports/<list_8> -->\n \n root/locations/<list_0>/transports/<list_8> \n \n 8 \n \n<!-- root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_8> -->\n \n root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_8> \n \n \n \n<!-- root/locations/<list_0>/transports/<list_1>/volume -->\n \n root/locations/<list_0>/transports/<list_1>/volume \n \n volume \n \n<!-- root/locations/<list_0>/transports/<list_1>->root/locations/<list_0>/transports/<list_1>/volume -->\n \n root/locations/<list_0>/transports/<list_1>->root/locations/<list_0>/transports/<list_1>/volume \n \n \n \n<!-- root/locations/<list_0>/transports/<list_1>/volume/-5:microliter -->\n \n root/locations/<list_0>/transports/<list_1>/volume/-5:microliter \n \n -5:microliter \n \n<!-- root/locations/<list_0>/transports/<list_1>/volume->root/locations/<list_0>/transports/<list_1>/volume/-5:microliter -->\n \n root/locations/<list_0>/transports/<list_1>/volume->root/locations/<list_0>/transports/<list_1>/volume/-5:microliter \n \n \n \n<!-- root/locations/<list_0>/transports/<list_4>/volume -->\n \n root/locations/<list_0>/transports/<list_4>/volume \n \n volume \n \n<!-- root/locations/<list_0>/transports/<list_4>->root/locations/<list_0>/transports/<list_4>/volume -->\n \n root/locations/<list_0>/transports/<list_4>->root/locations/<list_0>/transports/<list_4>/volume \n \n \n \n<!-- root/locations/<list_0>/transports/<list_4>/pump_override_volume -->\n \n root/locations/<list_0>/transports/<list_4>/pump_override_volume \n \n pump_override_volume \n \n<!-- root/locations/<list_0>/transports/<list_4>->root/locations/<list_0>/transports/<list_4>/pump_override_volume -->\n \n root/locations/<list_0>/transports/<list_4>->root/locations/<list_0>/transports/<list_4>/pump_override_volume \n \n \n \n<!-- root/locations/<list_0>/transports/<list_4>/volume/-5:microliter -->\n \n root/locations/<list_0>/transports/<list_4>/volume/-5:microliter \n \n -5:microliter \n \n<!-- root/locations/<list_0>/transports/<list_4>/volume->root/locations

\n<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"\n "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">\n<!-- Generated by graphviz version 2.40.1 (20161225.0304)\n -->\n<!-- Title: %3 Pages: 1 -->\n \n \n %3 \n \n<!-- root/locations -->\n \n root/locations \n \n locations \n \n<!-- root/locations/<list_0> -->\n \n root/locations/<list_0> \n \n 0 \n \n<!-- root/locations->root/locations/<list_0> -->\n \n root/locations->root/locations/<list_0> \n \n \n \n<!-- root/locations/<list_0>/transports -->\n \n root/locations/<list_0>/transports \n \n transports \n \n<!-- root/locations/<list_0>->root/locations/<list_0>/transports -->\n \n root/locations/<list_0>->root/locations/<list_0>/transports \n \n \n \n<!-- root/locations/<list_0>/transports/<list_3> -->\n \n root/locations/<list_0>/transports/<list_3> \n \n 3 \n \n<!-- root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_3> -->\n \n root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_3> \n \n \n \n<!-- root/locations/<list_0>/transports/<list_4> -->\n \n root/locations/<list_0>/transports/<list_4> \n \n 4 \n \n<!-- root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_4> -->\n \n root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_4> \n \n \n \n<!-- root/locations/<list_0>/transports/<list_5> -->\n \n root/locations/<list_0>/transports/<list_5> \n \n 5 \n \n<!-- root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_5> -->\n \n root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_5> \n \n \n \n<!-- root/locations/<list_0>/transports/<list_6> -->\n \n root/locations/<list_0>/transports/<list_6> \n \n 6 \n \n<!-- root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_6> -->\n \n root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_6> \n \n \n \n<!-- root/locations/<list_0>/transports/<list_3>/mode_params -->\n \n root/locations/<list_0>/transports/<list_3>/mode_params \n \n mode_params \n \n<!-- root/locations/<list_0>/transports/<list_3>->root/locations/<list_0>/transports/<list_3>/mode_params -->\n \n root/locations/<list_0>/transports/<list_3>->root/locations/<list_0>/transports/<list_3>/mode_params \n \n \n \n<!-- root/locations/<list_0>/transports/<list_3>/mode_params/tip_position -->\n \n root/locations/<list_0>/transports/<list_3>/mode_params/tip_position \n \n tip_position \n \n<!-- root/locations/<list_0>/transports/<list_3>/mode_params->root/locations/<list_0>/transports/<list_3>/mode_params/tip_position -->\n \n root/locations/<list_0>/transports/<list_3>/mode_params->root/locations/<list_0>/transports/<list_3>/mode_params/tip_position \n \n \n \n<!-- root/locations/<list_0>/transports/<list_3>/mode_params/tip_position/position_z -->\n \n root/locations/<list_0>/transports/<list_3>/mode_params/tip_position/position_z \n \n position_z \n \n<!-- root/locations/<list_0>/transports/<list_3>/mode_params/tip_position->root/locations/<list_0>/transports/<list_3>/mode_params/tip_position/position_z -->\n \n root/locations/<list_0>/transports/<list_3>/mode_params/tip_position->root/locations/<list_0>/transports/<list_3>/mode_params/tip_position/position_z \n \n \n \n<!-- root/locations/<list_0>/transports/<list_3>/mode_params/tip_position/position_z/detection -->\n \n root/locations/<list_0>/transports/<list_3>/mode_params/tip_position/position_z/detection \n \n detection \n \n<!-- root/locations/<list_0>/transports/<list_3>/mode_params/tip_position/position_z->root/locations/<list_0>/transports/<list_3>/mode_params/tip_position/position_z/detection -->\n \n root/locations/<list_0>/transports/<list_3>/mode_params/tip_position/position_z->root/locations/<list_0>/transports/<list_3>/mode_params/tip_position/position_z/detection \n \n \n \n<!-- root/locations/<list_0>/transports/<list_3>/mode_params/tip_position/position_z/detection/method -->\n \n root/locations/<list_0>/transports/<list_3>/mode

\n<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"\n "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">\n<!-- Generated by graphviz version 2.40.1 (20161225.0304)\n -->\n<!-- Title: %3 Pages: 1 -->\n \n \n %3 \n \n<!-- root/locations -->\n \n root/locations \n \n locations \n \n<!-- root/locations/<list_0> -->\n \n root/locations/<list_0> \n \n 0 \n \n<!-- root/locations->root/locations/<list_0> -->\n \n root/locations->root/locations/<list_0> \n \n \n \n<!-- root/locations/<list_0>/transports -->\n \n root/locations/<list_0>/transports \n \n transports \n \n<!-- root/locations/<list_0>->root/locations/<list_0>/transports -->\n \n root/locations/<list_0>->root/locations/<list_0>/transports \n \n \n \n<!-- root/locations/<list_0>/transports/<list_0> -->\n \n root/locations/<list_0>/transports/<list_0> \n \n 0 \n \n<!-- root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_0> -->\n \n root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_0> \n \n \n \n<!-- root/locations/<list_0>/transports/<list_3> -->\n \n root/locations/<list_0>/transports/<list_3> \n \n 3 \n \n<!-- root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_3> -->\n \n root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_3> \n \n \n \n<!-- root/locations/<list_0>/transports/<list_4> -->\n \n root/locations/<list_0>/transports/<list_4> \n \n 4 \n \n<!-- root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_4> -->\n \n root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_4> \n \n \n \n<!-- root/locations/<list_0>/transports/<list_5> -->\n \n root/locations/<list_0>/transports/<list_5> \n \n 5 \n \n<!-- root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_5> -->\n \n root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_5> \n \n \n \n<!-- root/locations/<list_0>/transports/<list_6> -->\n \n root/locations/<list_0>/transports/<list_6> \n \n 6 \n \n<!-- root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_6> -->\n \n root/locations/<list_0>/transports->root/locations/<list_0>/transports/<list_6> \n \n \n \n<!-- root/locations/<list_0>/transports/<list_0>/mode_params -->\n \n root/locations/<list_0>/transports/<list_0>/mode_params \n \n mode_params \n \n<!-- root/locations/<list_0>/transports/<list_0>->root/locations/<list_0>/transports/<list_0>/mode_params -->\n \n root/locations/<list_0>/transports/<list_0>->root/locations/<list_0>/transports/<list_0>/mode_params \n \n \n \n<!-- root/locations/<list_0>/transports/<list_0>/mode_params/tip_position -->\n \n root/locations/<list_0>/transports/<list_0>/mode_params/tip_position \n \n tip_position \n \n<!-- root/locations/<list_0>/transports/<list_0>/mode_params->root/locations/<list_0>/transports/<list_0>/mode_params/tip_position -->\n \n root/locations/<list_0>/transports/<list_0>/mode_params->root/locations/<list_0>/transports/<list_0>/mode_params/tip_position \n \n \n \n<!-- root/locations/<list_0>/transports/<list_0>/mode_params/tip_position/position_z -->\n \n root/locations/<list_0>/transports/<list_0>/mode_params/tip_position/position_z \n \n position_z \n \n<!-- root/locations/<list_0>/transports/<list_0>/mode_params/tip_position->root/locations/<list_0>/transports/<list_0>/mode_params/tip_position/position_z -->\n \n root/locations/<list_0>/transports/<list_0>/mode_params/tip_position->root/locations/<list_0>/transports/<list_0>/mode_params/tip_position/position_z \n \n \n \n<!-- root/locations/<list_0>/transports/<list_0>/mode_params/tip_position/position_z/offset -->\n \n root/locations/<list_0>/transports/<list_0>/mode_params/tip_position/position_z/offset \n \n offset \n \n<!-- root/locations/<list_0>/transports/<list_0>/mode_params/tip_position/position_z->root/locations/<list_0>/transports/<list_0>/mode_params/tip_position/position_z/offset -->\n \n root/locations/<list_0>/transports/<list_0>/mode_para